In [ ]:
import os

# Set up robot either for virtual robot or subip of real robot..
virtualRobot = False
virtualPort = 50513

robot_subip = 41

if virtualRobot:
    os.chdir('c:\\GitDev\\dfo\\data')
else:
    os.chdir('../data')
    
cwd = os.getcwd()
print(cwd)

In [ ]:
import pandas as pd
import datetime
import csv
import math

class Robot:
    """ This class will manage where the robot is up to in a conversation and decide what action needs 
        to be taken.  The response from the human will be parsed and this will determine next actions, either
        responding or navigating. 
    """

    # Different states the robot is in.
    # Starts out as Waiting (for a greeting) and will then move into 
    # either Greeting help where it will ask for item, size
    # Can then navigate to item and repeat asking what else they want
    Waiting = "Waiting"
    GreetingHelp = "Greeting Help"
    GreetingNoHelp = "Greeting No Help"
    ItemFound = "Item Found"
    ItemNotFound = "Item Not Found"
    ItemOutStock = "Item Out Of Stock"
    NavigateToItem = "Navigate"
    ArrivedItem = "Arrived"
    ItemSize = "Item Size"
    ItemNoSize = "Item No Size"
    Goodbye = "Goodbye"
    ItemSearch = "Item Search"
    Exit = "Exit"
    
    # Item/Size Statuses
    noStatus = 0
    inStock = 1
    outStock = 2
    notFound = 3
    
    # Item Searches.  Will move from first time to searching and then to next item
    firstTime = 0
    searching = 1
    nextItem = 2
    
    # non Items are common items not stocked here
    nonItems = ['sunglasses', 'shoes', 'hat']
    colour_list = ['blue', 'black', 'yellow', "green"] 
    yesNo = ['yes', 'no']
    # Make text large on the screen and a 2 column list if required.
    style_list = """<style>
ul li{
font-size:50px;
} {
  columns: 2;
  -webkit-columns: 2;
  -moz-columns: 2;
}
h1 {font-size:80px;
}
</style>"""

    # Import the data to drive the interactions
    stock = pd.read_csv('stock.csv')       # Investory levels 
    itemsFile = pd.read_csv('items.csv')   # Description of items
    speechFile = pd.read_csv('speech.csv') # varied sentences the robot can say in each state
    pathing = pd.read_csv('path.csv')      # Navigation steps for getting from item to item, or back to origin

    # How much stock each item has
    total_stock = stock.groupby('item')['stock'].sum()
    
    # Build text of items in stock, not in stock, sizes and colours
    items_instock = " ".join([str(x) for x in total_stock[total_stock>0].index.values])
    items_tablet = style_list + '<h1>We have:</h1><div><ul class="columns" data-columns="2">'+ \
                  " ".join(['<li>{0}</li>'.format(itemsFile[(itemsFile["item"]== x)]["plural"].values[0].title()) for x in total_stock[total_stock>0].index.values]) + \
                  '</ul></div>'
    items_outstock = " ".join([str(x) for x in total_stock[total_stock==0].index.values])
    size_list = " ".join(['"{0}"'.format(x) for x in stock['size'].unique()])
    colours = " ".join(['"{0}"'.format(x) for x in colour_list])
    # Create stock content that is "{colours} {sizes} {colours} item1" "{colours} {sizes} {colours} item1"  .. "{colours} {sizes} {colours} itemn"
    # So a response could be jeans, blue jeans, blue large jeans or large blue jeans
    stock_list = ""
    for x in total_stock.index.values:
        stock_list += ' "{ ' + colours + ' } { ' + size_list + ' } { ' + colours + ' } ' + x + '"'
    nonitemString = " ".join([str(x) for x in nonItems])
    greetString = "hello hi hey"                # Saying hello
    goodbyeString = 'goodbye "see ya" bye exit' # Saying goodbye
    yesString = "yes maybe perhaps sure yeah"   # synonyms for yes
    noString = "no nope nah nothing"            # synonyms for no
    hello_tablet = style_list + "<h1 align='center'>Say 'Hello' to start!</h1>"
    help_tablet = style_list + "<h1 align='center'>Do you need some assistance?</h1>"

    # Attributes maintained by the robot object
    responseExpected = False # Is the human expected to say something
    response = ""            # last response heard 
    textToSay = ""           # Next line for the robot to say
    Action = ""              # Action to take
    sayLine = False  #       # Is the robot expected to speak
    faceDetected = False     # Has a face been seen
    realRobot = False        # Is this a Pepper robot or simulation
    item = ""                # Current item the user is interested in
    itemStatus = noStatus    # Status of item
    size = ""                # Size user is looking for
    discount = 0             # Discount of the item
    plural = ""              # Plural of the item for speaking  - so belts is plural of belt
    sizeStatus = noStatus    # Status of size
    colour = ""              # Colour being searched for
    content = greetString    # Dialog content loaded to robot
    tabletDisplay = hello_tablet   # Tablet content loaded to robot
    exit = False                   # Is the robot finished
    itemSearch = firstTime         # Is this the first time this user is looking for something
    # Dataframe for recording interactions
    dfActions = pd.DataFrame(columns=['Who', 'Action', 'Time', 'State', 'Speech', 'Item', 'Size'])
    currentLocation = 'origin'     # Current location of robot, starts at front of store (origin)
    newLocation = ''               # New location to navigate to
    moveRequired = False           # Is a move required to get to the new location
    
    def __init__(self):
        """ Start are Waiting for the human to say hello """
        self.Action = self.Waiting
        
    # Write out record of interactions for later analysis
    def writeInteractions(self):
        """ Save the actions dataframe to disk once the robot stops.  This can be used for later analysis. """
        
        filename = "Interaction_" + datetime.datetime.now().strftime("%Y%m%d%H%M%S") + ".csv"
        self.dfActions.to_csv(filename, index=True, sep=",", quoting=csv.QUOTE_NONNUMERIC)
    
    
    def create_content(self):
        """ Create the dialog content to be loaded onto the robot.  Dependent on where the conversation is up to. """
        
        if self.item == "":
            self.content = self.stock_list + " " + self.nonitemString + " " + self.goodbyeString + " " + self.noString
        elif self.size == "":
            self.content = self.size_list + " " + self.goodbyeString + " " + self.noString
      #  Ignore colour for now..
      #  elif self.colour == "":
      #      self.content = self.colour_list + " " + self.goodbyeString
        # Yes / No / Goodbye response
        else:
            self.content = self.yesString + " " + self.noString + " " + self.goodbyeString
    
    # Parse the response captured by speech recognition.  this could be multiple words
    # eg extra large jeans...
    def parse_response(self):
        """ Parse what the human has said.  There could  e multiple elements such as Size and Item. """
    
        # Split into words
        split_list = self.response.split()
        
        # If we are looking for item, it will be the final word in the string
        if self.item == "":
            self.item = split_list.pop(-1)
            # Find the plural of the item for repeating back to user..
            self.plural = self.itemsFile[(self.itemsFile["item"]== self.item)]["plural"].values[0]
            
            # Is this item either in stock, completely out of stock or not sold here
            if self.item in self.total_stock[self.total_stock>0].index.values:
                self.itemStatus = self.inStock
            elif self.item in self.total_stock[self.total_stock==0].index.values:
                self.itemStatus = self.outStock
            elif self.item in self.nonItems:
                self.itemStatus = self.notFound
            
        # Then move onto looking for a colour and remove from string
        for word in self.colour_list:
            if word in split_list:
                self.colour = word
                split_list.remove(word)  
                    
        # Finally, what is left should be the size which could be 1 or 2 words, so 
        # put the list back into a string and look for a size term
        sizeString = " ".join([str(x) for x in split_list])
        for word in self.stock['size'].unique():
            if word == sizeString:
                # found a word, stop looking
                self.size = word
                break
                    
        # Check if we have this item and size in stock
        if self.size != "":
            if self.itemStatus == self.inStock:
               if self.stock[(self.stock["size"]== self.size) & (self.stock["item"]== self.item)]["stock"].values > 0:
                   # Set the discount to tell the human..
                   self.discount = self.stock[(self.stock["size"]== self.size) & (self.stock["item"]== self.item)]["discount"].values[0]
                   self.sizeStatus = self.inStock
               else:
                   self.sizeStatus = self.outStock
            else:
               self.sizeStatus = self.noStatus
        else:
            self.sizeStatus = self.noStatus
            
    
    # There can be a number of sentences contained in the speech file that say the same thing, just in varied ways
    # So look up the datafrome on Phase, item, item status and size status and randomly pick one of the sentences
    # that match.  This will then be said by the robot with possible terms added (the item for instance)
    def speechText(self):
        """ This method finds a sentence for the robot to say.  There can be a number of sentences contained in the
            speech file that say the same thing, just in varied ways.  So look up the datafrome on Phase, item, 
            item status and size status and randomly pick one of the sentences that match.  This will then be said 
            by the robot with possible terms added (the item for instance). 
        """
        
        
        return self.speechFile[(self.speechFile["Phase"]==self.Action) & 
                                 (self.speechFile["itemSearch"]==self.itemSearch) &
                                 (self.speechFile["itemStatus"]==self.itemStatus) &
                                 (self.speechFile["sizeStatus"]==self.sizeStatus)]["Speech"].sample(n=1).values[0]
        
    def decideAction(self, actor="Robot", action="Speak"):
        """ Decide what the robot will say and do based on the state it is in and the reponses it has received 
            so far.  """

        print(str(self.sizeStatus) + str(self.itemStatus) + str(self.itemSearch) + self.Action)

        # Processing will give the robot a sentence to say, and load the new words into a dialog
        # that it will be looking for
        # This can be expanded to set tablet output..
        if self.Action == self.Waiting:
            self.textToSay = self.speechText()
            self.tabletDisplay = self.help_tablet
            self.sayLine = True
            self.responseExpected = True
            self.faceDetected = True
            self.itemSearch = self.firstTime
            self.content = self.yesString + " " + self.noString + " " + self.goodbyeString
            
        elif self.Action == self.GreetingHelp:
            # Clear attributes so that it is starting again for asking for what the user wants
            if self.itemSearch == self.firstTime:
               self.textToSay = self.speechText()
               self.itemSearch = self.searching
               self.tabletDisplay = self.items_tablet
               self.item = ""
               self.size = ""
               self.colour = ""
               self.itemStatus = self.noStatus
               self.sizeStatus = self.noStatus
            # Clear attributes so that it is starting again for asking for what the user wants
            elif self.itemSearch == self.nextItem:
               self.textToSay = self.speechText()
               self.tabletDisplay = self.items_tablet
               self.itemSearch = self.searching
               self.item = ""
               self.size = ""
               self.colour = ""
               self.itemStatus = self.noStatus
               self.sizeStatus = self.noStatus                
            else:            
               # Item is completely out of stock
               if self.itemStatus == self.outStock:
                   self.textToSay = self.speechText().format(self.plural)
                   self.item = ""
                   self.size = ""
                   self.colour = ""
                   self.itemStatus = self.noStatus
                   self.sizeStatus = self.noStatus
               # Item is not sold here
               elif self.itemStatus == self.notFound:
                   self.textToSay = self.speechText().format(self.plural)
                   self.item = ""
                   self.size = ""
                   self.colour = ""
                   self.itemStatus = self.noStatus
                   self.sizeStatus = self.noStatus
                    
               # Item and size are in stock
               elif self.itemStatus == self.inStock and self.sizeStatus == self.inStock:
                   self.textToSay = self.speechText().format(self.size, self.plural, self.discount)
                   self.Action = self.ItemFound
                    
               # Item is in stock, now to get size from customer
               elif self.itemStatus == self.inStock and self.sizeStatus == self.noStatus:
                   self.textToSay = self.speechText().format(self.plural)
                   self.tabletDisplay = self.style_list + '<h1>' + self.item.capitalize() + ' sizes are:</h1><div><ul class="columns" data-columns="2">'+ \
                   " ".join(['<li>{0}</li>'.format(x.title()) for x in self.stock[(self.stock["item"] == self.item) & 
                                                                          (self.stock["stock"]>0)]['size'].unique()]) + \
                   '</ul></div>'
                
               # Item is available but not in the size they are looking for
               elif self.itemStatus == self.inStock and self.sizeStatus == self.outStock:
                   self.textToSay = self.speechText().format(self.size, self.plural)
                   self.size = ""
                   self.sizeStatus = self.noStatus
            
            # Robot will say the speech and expect a response
            self.sayLine = True
            self.responseExpected = True
            self.create_content()  # Create the list of words to be loaded to the Dialog
            
        elif self.Action == self.GreetingNoHelp:
            self.itemStatus = self.noStatus
            self.sizeStatus = self.noStatus
            self.itemSearch = self.firstTime
            self.textToSay = self.speechText() 
            self.tabletDisplay = self.hello_tablet
            self.sayLine = True
            self.responseExpected = False
            self.content = self.greetString + " " + self.goodbyeString
            self.newLocation = 'origin'
            # If robot moved then return to origin, conversation is over
            if self.currentLocation != self.newLocation:
                RobotAction.moveRequired = True
                
        elif self.Action == self.NavigateToItem:
            self.textToSay = self.speechText()
            self.sayLine = True
            self.responseExpected = False
            self.newLocation = self.item
            if self.currentLocation != self.newLocation:
                RobotAction.moveRequired = True
            
        elif self.Action == self.Goodbye:
            self.itemStatus = self.noStatus
            self.sizeStatus = self.noStatus
            self.itemSearch = self.firstTime
            self.textToSay = self.speechText()
            self.tabletDisplay = self.hello_tablet
            self.sayLine = True
            self.Action = self.Waiting
            self.responseExpected = False
            self.faceDetected = False
            self.content = self.greetString + " " + self.goodbyeString
            self.newLocation = 'origin'
            if self.currentLocation != self.newLocation:
                RobotAction.moveRequired = True
                
        elif self.Action == self.Exit:
            self.textToSay = "Shutting down."
            self.sayLine = True
            self.responseExpected = False
            self.faceDetected = False
            self.newLocation = 'origin'
            if self.currentLocation != self.newLocation:
                RobotAction.moveRequired = True
                
        # Write to dataframe what the result is, to be saved for later analysis..
        self.dfActions = self.dfActions.append({'Who': actor, 'Action': action, 'Time': datetime.datetime.now(), \
                                                'State': self.Action, 'Speech': self.textToSay, \
                                                'Item': self.item, 'Size': self.size}, ignore_index=True)
            

    def UpdateState(self, userInput, actor="Human", action="Speak"):
        """" Update the state of the robot depending on the response that comes in. """
        
        print("update state" + userInput + self.Action)
        self.response = userInput
        # Save the current state and response.
        self.dfActions = self.dfActions.append({'Who': actor, 'Action': action, 'Time': datetime.datetime.now(), \
                                                'State': self.Action, 'Speech': userInput, \
                                                'Item': self.item, 'Size': self.size}, ignore_index=True)

        # User said hello, so robot will respond with greeting and ask if they need help
        if self.response in self.greetString:
            self.Action = self.Waiting
            
        # Time to shutdown
        elif self.response == "exit":
            self.Action = self.Exit
            self.responseExpected = False
            self.faceDetected = False
            self.sayLine = False
            self.exit = True
            self.itemSearch = self.noStatus 
            self.itemStatus = self.noStatus
            self.sizeStatus = self.noStatus
            
        # Return to the front of the store when the user says goodbye
        elif self.response == "goodbye":
            self.Action = self.Goodbye
            self.responseExpected = False
            self.faceDetected = False
            self.sayLine = False
            
        # User does not want help, go back to waiting for the next interaction
        elif self.Action == self.GreetingNoHelp:
            self.Action = self.Waiting
            self.faceDetected = False
            
        # Robot greeted the user and asked if they needed help so check if yes or no
        elif self.Action == self.Waiting:            
            if self.response in  self.yesString:
                self.Action = self.GreetingHelp
            elif self.response in self.noString:
                self.Action = self.GreetingNoHelp
                
        # User wants help so parse response
        elif self.Action == self.GreetingHelp:
            if self.response in self.noString:
                self.Action = self.GreetingNoHelp
            else:
                self.parse_response()            
                
        # Robot has asked if user wants to go to the location of the item
        elif self.Action == self.ItemFound:
            if self.response in self.yesString:
                self.Action = self.NavigateToItem
            elif self.response in self.noString:
                self.Action = self.GreetingHelp
                self.itemSearch = self.nextItem 
                self.itemStatus = self.noStatus
                self.sizeStatus = self.noStatus

        # Navigating to item. Once there will ask what next they are looking for
        elif self.Action == self.NavigateToItem:
            self.Action = self.GreetingHelp
            self.itemSearch = self.nextItem # Save in previous items list
            self.itemStatus = self.noStatus
            self.sizeStatus = self.noStatus
            
        # User said goodbye, go back to waiting for an interaction.
        elif self.Action == self.Goodbye:
            self.Action = self.Waiting
        else:
            self.Action = self.Waiting

print("Success")

In [ ]:
# This sets up the session either to the Pepper robot or choreographe virtual robot, turning on
# services as needed.
import qi

# Create object session
s = qi.Session()
# Set above to determine whether to connect to Choregraphe or actually to the robot.
# Once set up here, the next cell works on either the real robot or simulated one.
if virtualRobot:
    robot_ip = "localhost"
    port_num = virtualPort
else:    
    robot_ip = '192.168.1.'+str(robot_subip)
    port_num = 9559

s.connect('tcp://' + robot_ip + ':'+str(port_num))
print('tcp://' + robot_ip + ':'+str(port_num))

dialog = s.service('ALDialog')
memory = s.service('ALMemory')
tts = s.service('ALTextToSpeech')
motion_proxy = s.service("ALMotion")

# start face detection, speech recognition and tablet
# only if on a real robot
if not virtualRobot:
   sr = s.service('ALSpeechRecognition')
   fd = s.service("ALFaceDetection")
   tablet = s.service("MagicTablet")
   video_proxy = s.service("ALVideoDevice")
else:
   sr = s.service('ALDialog')

# Get robot ready to move
posture = s.service('ALRobotPosture')
posture.goToPosture("Stand", 0.5)

print("Ready")

In [ ]:
# Once the above cells have been run, the robot is ready to controlled.  So this 
# cell does the work of detecting 
import time
import numpy as np
import face_recognition as fr
import cv2
import matplotlib.pyplot as plt

def get_frame(proxy, camera_idx=0, resolution_idx=1, colorspace_idx=11, fps=20):
    """ Take a picture from the robot, to process for face detection. """
    
    if not proxy.isCameraOpen(camera_idx):
        proxy.openCamera(camera_idx)

    if not proxy.isCameraStarted(camera_idx):
        video_proxy.startCamera(camera_idx)
    
    if resolution_idx in [3, 4]:
        # max fps for these resolutions
        fps = 1
    
    sub = proxy.subscribeCamera(
        "get_frame_sub",
        camera_idx,
        resolution_idx,
        colorspace_idx,
        fps
    )
    
    np_image = None
    
    try:
        timeout = 3
        start_time = time.time()
        result = None
        while time.time() - start_time < timeout and result is None:
            result = proxy.getImageRemote(sub)

        if result:
            buffer_image = result[6]
            
            if result[2] == 3:
                img_shape = (result[1], result[0], result[2])
                im_format = np.uint8
            else:
                img_shape = (result[1], result[0])
                im_format = np.uint16
            
            temp = np.frombuffer(buffer_image, im_format)
            np_image = np.reshape(temp, img_shape)
    except Exception as e:
        print(e)
    finally:
        proxy.unsubscribe(sub)
    
    return np_image

def get_depth(depth_im, bbox, estim_func=np.median):
    """ Get the distance to the face detected. """
    
    probe = depth_im[bbox[0]:bbox[2], bbox[3]:bbox[1]]
    depth_val = estim_func(probe) / 1000.0
    
    return depth_val

def detect_faces(video_proxy, debug_color=(0, 0, 255)):
    """ Detect if there are any faces in the frame taken by the robot."""
    
    current_frame = get_frame(video_proxy)
    current_depth = get_frame(video_proxy, camera_idx=2, colorspace_idx=17)
    faces = fr.face_locations(current_frame)

    debug_image = current_frame.copy()
    output_list = []
    for face_id, bbox in enumerate(faces):
        cur_depth = get_depth(current_depth, bbox)
        output_list.append((bbox, cur_depth))
        cur_top = (bbox[3], bbox[0])
        cur_bottom = (bbox[1], bbox[2])
        cv2.rectangle(debug_image, cur_top, cur_bottom, debug_color, 2)
        cv2.putText(
            debug_image,
            '{0}'.format(cur_depth), 
            (cur_top[0], max(0, cur_top[1]-5)), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5,
            debug_color,
            2
        )
    
    return output_list, debug_image 


def buildTopic(content):
    """ Build a dialog topic, to load as what the robot will be listening for."""

    return '''topic: ~test_content()\n
              language: enu\n
              concept:(options) [ {0} ]\n
              u: (_~options) $response=$1\n'''.format(content)

def stop_listening(dialog_proxy, sr_proxy):
    """ Unsubscribe from listening services and topics before starting."""
    
    subscribers = dialog_proxy.getSubscribersInfo()
    for sub in subscribers:
        dialog_proxy.unsubscribe(sub[0])
        
    subscribers = sr_proxy.getSubscribersInfo()
    for sub in subscribers:
        sr_proxy.unsubscribe(sub[0])

    active_topics = dialog_proxy.getActivatedTopics()
    
    for topic in active_topics:
        dialog_proxy.deactivateTopic(topic)

    loaded_topics = dialog_proxy.getAllLoadedTopics()
    for topic in loaded_topics:
        dialog_proxy.unloadTopic(topic)
        
def Respond(response, actor="Human", action="Move"):
    """ Process input and decide next action, Called either from Speech Recog, Face Detected or Navigating."""

    RobotAction.UpdateState(response) 
    RobotAction.decideAction()

    dialog.deactivateTopic('test_content') 
    dialog.unloadTopic('test_content') 

    dialog.loadTopicContent(buildTopic(RobotAction.content))
    
    if RobotAction.sayLine: 
        if not virtualRobot:
            tablet.html(RobotAction.tabletDisplay, {})
        tts.say(RobotAction.textToSay) 
    dialog.activateTopic('test_content')

# Speech Recognition callback.  Process what the robot heard and then move on to the 
# next state.
def speech_recog(value):
    """ Speech Recognition callback.  Process what the robot heard and then move on to the next state."""

    print("Speech Recog " + value) 
    response = memory.getData("response") 
    print(response)

    # Word was recognized so determine if it was the right word or not.
    #Then move on to the next conversation piece.
    Respond(response)

# Navigate from one item to the next
# Location to and From set in RobotAction and then the actual movement done here.
def navigator():
    """ Navigate to the new location, based on the steps in the path file."""
    
    columns = RobotAction.pathing[(RobotAction.pathing["start"]== RobotAction.currentLocation) & (RobotAction.pathing["dest"]== RobotAction.newLocation)]
    for index, row in columns.iterrows():
         x = row["x"]
         y = row["y"]
         theta = row["theta"]*math.pi/180
         print(str(x) + " " + str(y) + " " + str(theta))
         motion_proxy.moveTo(x, y, 0) #(forward (x), left, right)
         print(motion_proxy.getRobotPosition(True))
    
    print("moved from " + RobotAction.currentLocation + " to " + RobotAction.newLocation)
    RobotAction.currentLocation = RobotAction.newLocation  # Update current location
    RobotAction.moveRequired = False  # Do not move until next navigation needed..
    
    
# Initialise a robot object 
        
# Set a threshold for the distance from user's face to robot
depth_threshold = 1.5  
RobotAction = Robot()
RobotAction.realRobot = True
stop_listening(dialog, sr)
if not virtualRobot:
   tablet.html(RobotAction.tabletDisplay, {})
      
# Create an ALDialog topic to listen for.
topic = ('''topic: ~test_content()\n 
            language: enu\n 
            concept:(options) [ ''' + RobotAction.content + ''' ]\n 
            u: (_~options) $response=$1\n''') 

# Load the new topic
dialog.loadTopicContent(topic)
    
# Activate the topic and subscribe to user speech
dialog.activateTopic("test_content")
sub_id = dialog.subscribe("my_test")
subscriber = memory.subscriber('Dialog/MatchedInput')
subscriber.signal.connect(speech_recog)

# before starting listening to the user we need a memory variable
# that's the same we are using in our topic
memory.insertData("response", "")
    
# now let's wait for the end of the dialog
# Run for 60 seconds to ensure we do not get carried away..
max_time = 500
time_start = time.time()
while time.time() - time_start < max_time:
    
    # Face detected so Robot to initiate conversation
    if not RobotAction.faceDetected:
        face_list, debug_img = detect_faces(video_proxy)
        print(face_list)
        if (len(face_list) == 0): # if the list is empty the print no face
            print("No one is there")
        elif face_list[0][1] < depth_threshold: # if the list print out depth, then check the threshold
            Respond('hello', actor='Robot') # Robot to start conversation

    # Trigger moving and then speak next bit..
    if RobotAction.moveRequired == True:
        navigator()
        if RobotAction.Action != RobotAction.Waiting:
           Respond('navigate', actor="Robot", action="Move")  
            
    try:
        if RobotAction.exit:
            break            
    except RunTimeError as e:
        print(str(e))
    
    time.sleep(0.5)

# we can stop listening now
stop_listening(dialog, sr)

tablet.show(tablet.animation("LOGO"), [], [])  # Show logo once all finished.
RobotAction.writeInteractions()  # Save the interactions for later processing.

In [ ]:
# This is for testing outside of real and virtual robots, testing that the dialog works and the 
# state is correctly updated each time through the interaction.

import csv

# Navigate from one item to the next
def navigator():
    columns = RobotAction.pathing[(RobotAction.pathing["start"]== RobotAction.currentLocation) & (RobotAction.pathing["dest"]== RobotAction.newLocation)]
    for index, row in columns.iterrows():
         x = row["x"]
         y = row["y"]
         theta = row["theta"]*math.pi/180
         print(str(x) + " " + str(y) + " " + str(theta))
     #    motion_proxy.moveTo(x, y, 0) #(forward (x), left, right)
    
    
    print("moved from " + RobotAction.currentLocation + " to " + RobotAction.newLocation)
    RobotAction.currentLocation = RobotAction.newLocation
    RobotAction.moveRequired = False
    
    
# Run the dialog without Choreographe...
RobotAction = Robot()
RobotAction.realRobot = False
response = ""

RobotAction.decideAction()
while response != "exit":

  # Get user input
  if not RobotAction.realRobot:
     if RobotAction.responseExpected:
        response = raw_input(RobotAction.textToSay).lower()
     else:
        print("text" + RobotAction.textToSay)
        response = ""

  print(response)
  RobotAction.UpdateState(response)
  RobotAction.decideAction()
  if RobotAction.moveRequired == True:   
      navigator()
      RobotAction.UpdateState('navigate', actor="Robot", action="Move") 
      RobotAction.decideAction()
    
  print("State = " + str(RobotAction.Action))

print("Finished!")
RobotAction.writeInteractions()

In [ ]:
RA = Robot()
print(RA.pathing)